In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 7.1MB/s 
     |████████████████████████████████| 460kB 33.1MB/s 
     |████████████████████████████████| 92kB 5.7MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls -l /content/drive/My\ Drive/Colab\ Notebooks/datasets/naverRatings.zip

-rw------- 1 root root 7903524 Mar 31 02:15 '/content/drive/My Drive/Colab Notebooks/datasets/naverRatings.zip'


In [5]:
!unzip /content/drive/My\ Drive/Colab\ Notebooks/datasets/naverRatings.zip

Archive:  /content/drive/My Drive/Colab Notebooks/datasets/naverRatings.zip
  inflating: naverRatings.txt        


In [6]:
!ls -l naverRatings.txt

-rw-r--r-- 1 root root 19515078 May  4  2020 naverRatings.txt


In [7]:
train_data = pd.read_table('naverRatings.txt')
train_data[:10]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
5,2190435,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1
6,9279041,완전 감동입니다 다시봐도 감동,1
7,7865729,개들의 전쟁2 나오나요? 나오면 1빠로 보고 싶음,1
8,7477618,굿,1
9,9250537,바보가 아니라 병 쉰 인듯,1


In [8]:
print(len(train_data))

200000


In [9]:
print(train_data.isnull().values.any())

True


In [10]:
train_data = train_data.dropna(how='any')
print(train_data.isnull().values.any())

False


In [11]:
print(len(train_data))

199992


In [12]:
train_data['document'] = train_data['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]','')
train_data[:5]

,id,document,label
0,8112052,어릴때보고지금다시봐도재밌어요ㅋㅋ,1
1,8132799,디자인을배우는학생으로외국디자이너와그들이일군전통을통해발전해가는문화산업이부러웠는데사실우...,1
2,4655635,폴리스스토리시리즈는부터뉴까지버릴께하나도없음최고,1
3,9251303,와연기가진짜개쩔구나지루할거라고생각했는데몰입해서봤다그래이런게진짜영화지,1
4,10067386,안개자욱한밤하늘에떠있는초승달같은영화,1


In [13]:
train_data['document'].shape

(199992,)

In [14]:
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
%%time

from konlpy.tag import Okt

okt = Okt()
tokenized_data =[]

for sentence in train_data['document']:
  temp_X = okt.morphs(sentence, stem = True)
  temp_X = [word for word in temp_X if not word in stopwords]
  tokenized_data.append(temp_X)

In [ ]:
print(len(tokenized_data), tokenized_data.shape)

In [ ]:
print('리뷰의 최대 길이 :',max(len(l) for l in tokenized_data))
print('리뷰의 평균 길이 :',sum(map(len, tokenized_data))/len(tokenized_data))

In [ ]:
plt.hist([len(s) for s in tokenized_data], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = tokenized_data, size=100,  # embedding 차원 = 100차원
                 windows=5, min_count=5, workers = 4, sg= 1 )

model.wv.vectors.shape

In [ ]:
model.wv.most_similar('이병헌')

In [ ]:
model.wv.most_similar('로맨스')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls -l /content/drive/My\ Drive/Colab\ Notebooks/datasets/ko_w2v.zip

In [ ]:
!unzip /content/drive/My\ Drive/Colab\ Notebooks/dataests/ko_w2v.zip

In [ ]:
!ls -l ko.bin

In [ ]:
import gensim

model = gensim.models.Word2Vec.load('ko.bin')

In [ ]:
model.wv.most_similar('사랑')

In [ ]:
model.wv.most_similar('봄')